In [183]:
import warnings 
warnings.filterwarnings("ignore")
from sklearn.linear_model import ElasticNet

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder , PolynomialFeatures
from sklearn.linear_model import LinearRegression ,ElasticNetCV
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import train_test_split ,cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor , GradientBoostingRegressor
from scipy.stats import skew
train = pd.read_csv('/Users/hajiaga/Documents/house-prices-advanced-regression-techniques-2/train.csv')
train["SalePrice"] = np.log1p(train['SalePrice'])
test = pd.read_csv('/Users/hajiaga/Documents/house-prices-advanced-regression-techniques-2/test.csv')
sample_submission = pd.read_csv('/Users/hajiaga/Documents/house-prices-advanced-regression-techniques-2/sample_submission.csv')

In [184]:

data = pd.concat([train , test] , axis= 0)
data.drop(["Id"] ,axis=1, inplace=True)
data["Age"] = data["YrSold"] - data["YearBuilt"]
data["yaxsi"] = data["1stFlrSF"]+data["2ndFlrSF"]-data["LowQualFinSF"]
data["umumi"] = data["1stFlrSF"]+data["2ndFlrSF"]
data["LotFrontage"] = data.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
data['SaleType'] = data['SaleType'].fillna(data['SaleType'].mode()[0])
data['MoSold'] = data['MoSold'].astype(str)


In [185]:
str_cols = data.select_dtypes("O").columns
num_cols = data.select_dtypes("number").columns
num_cols = list(num_cols)
no_cols = ["MiscFeature" , "Fence" , "GarageCond" , "GarageQual" , "GarageFinish" , "FireplaceQu" , "BsmtFinType2" , "BsmtExposure", "BsmtCond" , "BsmtQual" , "Alley"]
for col in data.columns:
    if col != "SalePrice":
        if col in no_cols:
            data[col]=data[col].fillna("None")
        elif col in str_cols:
            data[col]=data[col].fillna(data[col].mode()[0])
        elif col in num_cols:
            data[col]=data[col].fillna(0)
np.array(no_cols)
data["TotalHouse"] = data["TotalBsmtSF"] + data["1stFlrSF"] + data["2ndFlrSF"]   
data["qualtot"] = data["TotalHouse"] * data["OverallQual"]


In [186]:
def detect_outliers(df,col):
    Q1 = df[col].quantile(0.005)
    Q3 = df[col].quantile(0.995)
    IQR = Q3-Q1
    lwr_bnd = Q1 - IQR*1.5
    hgh_bnd= Q3 + IQR*1.5
    df.drop(df[df[col]>hgh_bnd].index ,axis =0, inplace=True)
    df.drop(df[df[col]<lwr_bnd].index , axis = 0, inplace=True)


In [187]:
from sklearn.preprocessing import LabelEncoder
columns = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
for col in columns:
    le = LabelEncoder() 
    le.fit(list(data[col].values)) 
    data[col] = le.transform(list(data[col].values))




In [188]:
# ct = ColumnTransformer(transformers=[( "poly" ,PolynomialFeatures(degree=2), a.select_dtypes(include=['number']).columns)], remainder='passthrough')
# ct.fit_transform(data)


In [189]:
# cte = ColumnTransformer(transformers=[( "poly" ,PolynomialFeatures(degree=2), pd.Series((list(data.select_dtypes(include=['number']).columns).remove("SalePrice"))).astype("O"))], remainder='passthrough')
# cte.fit_transform(data)
# data.select_dtypes("number")
print(num_cols)


['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'YrSold', 'SalePrice', 'Age', 'yaxsi', 'umumi']


In [190]:
num_cols = data.dtypes[data.dtypes != "object"].index
skewed_feats = data[num_cols].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness = skewness[abs(skewness) > 0.75]
skewness.head()
from scipy.special import boxcox1p
skewed_features = skewness.index
print("SalePrice" in skewed_features)
lam = 0.15
for col in skewed_features:
    if col != "SalePrice":
        data[col] = boxcox1p(data[col] , lam)

data['Age'].fillna(data['Age'].median())

for col in data.columns:
    if data[col].isnull().sum()>0:
        print(col)
        print(data[col].isnull().sum())

True
SalePrice
1459


In [191]:

train=data[data['SalePrice'].notnull()]
for col in list(num_cols):
    detect_outliers(train,col)
test=data[data['SalePrice'].isnull()]
test.drop(['SalePrice'] , inplace=True , axis = 1)
num_cols = data.dtypes[data.dtypes == "object"].index
print(num_cols)

Index(['MSZoning', 'LandContour', 'Utilities', 'LotConfig', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation',
       'Heating', 'Electrical', 'GarageType', 'MiscFeature', 'SaleType',
       'SaleCondition'],
      dtype='object')


In [192]:
X=train.drop('SalePrice', axis=1)
Y=train['SalePrice']
num_cols = data.dtypes[data.dtypes == "object"].index

print(num_cols)

Index(['MSZoning', 'LandContour', 'Utilities', 'LotConfig', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation',
       'Heating', 'Electrical', 'GarageType', 'MiscFeature', 'SaleType',
       'SaleCondition'],
      dtype='object')


In [193]:
ct = ColumnTransformer([('encoder', OneHotEncoder(handle_unknown='ignore'), X.select_dtypes(include=['object']).columns)], remainder='passthrough')
X = ct.fit_transform(X)

X_train , X_test , Y_train , Y_test = train_test_split(X , Y , test_size=0.3 , random_state=42)
test= ct.transform(test)


In [194]:
def Poly_F(x , y , i , test ):
    pf = PolynomialFeatures(degree=i)
    pf.fit(x)
    x_poly = pd.DataFrame(pf.transform(x) , columns = pf.get_feature_names_out())
    pfe = PolynomialFeatures(degree=i)
    pfe.fit(test)
    test = pd.DataFrame(pfe.transform(test) , columns = pfe.get_feature_names_out())

    X_train , X_test , Y_train , Y_test = train_test_split(x_poly , y , test_size=0.3 , random_state=42)
    lr = LinearRegression()
    lr.fit(X_train , Y_train)
    # rmse_t = np.sqrt(mean_squared_error(Y_train , lr.predict(X_train)))
    # rmse= np.sqrt(mean_squared_error(Y_test , lr.predict(X_test)))
    return lr.predict(test)

In [195]:
# def enet(x , y , i , j):
#     enet = ElasticNet(alpha= i  , l1_ratio=j)
#     X_train , X_test , Y_train , Y_test = train_test_split(x , y , test_size=0.25 , random_state=42)
#     enet.fit(X_train , Y_train)
#     rmse_t = np.sqrt(mean_squared_error(Y_train , enet.predict(X_train)))
#     rmse= np.sqrt(mean_squared_error(Y_test , enet.predict(X_test)))
#     return rmse_t , rmse , i , j

In [196]:
# best_enet = {}
# best_enet = dict(best_enet)
# best_enety = {}
# best_enety = dict(best_enety)
# for i in [5,20,80]:
#     for j in [5,20,80]:
#         a , b , c , d= enet(X ,Y ,i/100 , j/100)
#         best_enet[f" i {c} , j {d}"] = b
#         best_enety[f" i {c} , j {d}"] = a


In [197]:
# print(print(min(best_enet, key=best_enet.get)))
# print(print(min(best_enety, key=best_enety.get)))
# print(best_enet )

In [198]:
#KNN - 0.39
#knn = KNeighborsClassifier()
#knn.fit(X_train , Y_train)
for col in data:
    if data[col].isnull().sum()/data[col].count()>0:
        print(col ,data[col].isnull().sum()/data[col].count() )

SalePrice 0.9993150684931507


In [199]:
#Poly degree 2
lr = LinearRegression()
lr.fit(X , Y)


LinearRegression()

In [200]:

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler 




In [201]:
# enet = ElasticNet(alpha=0.4,l1_ratio=0.9,max_iter=10000 )
# enet.fit(X , Y)
alpha = [0.005,0.0001,0.01,0.1,0.2,0.4,0.5]
l1 = [0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
modelcv = ElasticNetCV(l1_ratio=l1 , alphas = alpha , cv=10)
modelcv.fit(X,Y)
print(modelcv.alpha_ , modelcv.l1_ratio_)


0.005 0.05


In [202]:
ENet = make_pipeline(RobustScaler(), PCA() ,ElasticNet(alpha=modelcv.alpha_ , l1_ratio=modelcv.l1_ratio_, max_iter=20000 ))
enet = ENet.fit(X,Y)

In [203]:


filename = "/Users/hajiaga/Documents/house-prices-advanced-regression-techniques-2/sample_submission2.csv"
import os
test_pred =np.expm1(enet.predict(test))
sample_submission['SalePrice'] = test_pred
if os.path.exists(filename):
    os.remove(filename)
sample_submission.to_csv(filename, index=False) 
sample_submission.head()


,Id,SalePrice
0,1461,120219.312023
1,1462,160548.195690
2,1463,181818.427261
3,1464,197082.544768
4,1465,192503.394706
